## Load Packages and Read in Frame with Restaurant Names

In [1]:
#General Packages
import pandas as pd

#Scraping Packages
import requests
from urllib.request import urlopen as url
#import bs4
from bs4 import BeautifulSoup as soup

#Address Matching Package
from geopy import geocoders

In [16]:
#Read in a list of 2019 Mich Starred Restaurants in Bay Area
Mich_names = pd.read_excel("Mich Restaurants.xlsx")
names = Mich_names["Yelp_Name"].tolist()

## Scrape Restaurants' Business Info from Yelp 
### Data Scraped: address, price range, Yelp rating

In [17]:
def scrape_function(list):
    """Scrape Yelp info and put in csv"""    
    filename="dfaddress.csv"
    f=open(filename,"w")
    headers = "Yelp_Name, Rating, Price, Street, City, State"
    f.write(headers)
    for i in names: 
            my_url="https://www.yelp.com/biz/"+str(i)+"?start="
            r = requests.get(my_url)
            if r.status_code == 200: 
                request=url(my_url) 
                htmlscrap=request.read()
                request.close()
                page_soup=soup(htmlscrap,"html.parser")
                street = 'Unknown' if page_soup.find_all('span', {'itemprop': 'streetAddress'})[0].string is None else\
                page_soup.find_all('span', {'itemprop': 'streetAddress'})[0].string
                city = 'Unknown' if page_soup.find_all('span', {'itemprop': 'addressLocality'})[0].string is None else\
                page_soup.find_all('span', {'itemprop': 'addressLocality'})[0].string
                rating = 'Unknown' if page_soup.find_all('meta', {'itemprop': 'ratingValue'})[0].get('content') is None else\
                page_soup.find_all('meta', {'itemprop': 'ratingValue'})[0].get('content')
                price = 'Unknown' if page_soup.find_all('span', {'class': 'business-attribute price-range'})[0].string is None else\
                page_soup.find_all('span', {'class': 'business-attribute price-range'})[0].string
                state = 'Unknown' if page_soup.find_all('span', {'itemprop': 'addressRegion'})[0].string is None else\
                page_soup.find_all('span', {'itemprop': 'addressRegion'})[0].string
                f.write("\n" + str(i) +  "," + str(rating) + "," + str(price) + "," + str(street) + "," + str(city) + "," + str(state))
            else: 
                f.write("\n" + str(i))
    f.close()

In [18]:
scrape_function(names)

## Match Restaurant Addresses With GPS Coordinates

In [26]:
df = pd.read_csv("dfaddress.csv")
df["Address"] = df[' Street'] + ", " + df[' City'] + ", " + df[' State']

In [27]:
api_key = "Enter your API key here"
g = geocoders.GoogleV3(api_key=api_key)

In [28]:
#Match Restaurant Addresses With GPS Coordinates
loc_coordinates = []
loc_address = []

for address in df["Address"]:
    try:
        inputAddress = address
        location = g.geocode(inputAddress, timeout=15)
        loc_coordinates.append((location.latitude, location.longitude))
        loc_address.append(inputAddress)
    except:
        print('Error, skipping address...')

In [29]:
#Concatonate the two lists together creating a dataframe of address and coordinates.
df_geocodes = pd.DataFrame({'coordinate':loc_coordinates, 'Address':loc_address})

# Merge Frames for Final Dataframe Which Will be Read into Tableau 

In [30]:
Address = pd.merge(df, df_geocodes, how="left")

In [31]:
Full = Address.merge(Mich_names)

In [33]:
Full.to_csv("Full.csv")